<a href="https://colab.research.google.com/github/Nag112/aws-codepipeline-s3-codedeploy-linux/blob/master/elm_for_mnist_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical
from elm import ELM, load_model
import argparse
import os
import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument('--n_hidden_nodes', type=int, default=1024)
parser.add_argument('--loss',
    choices=['mean_squared_error', 'mean_absolute_error'],
    default='mean_squared_error',
)
parser.add_argument('--activation',
    choices=['sigmoid', 'identity'],
    default='sigmoid',
)

def softmax(x):
    c = np.max(x, axis=-1)
    upper = np.exp(x - c)
    lower = np.sum(upper, axis=-1)
    return upper / lower

def main(args):
    # ===============================
    # Load dataset
    # ===============================
    n_classes = 10
    (x_train, t_train), (x_test, t_test) = mnist.load_data()

    # ===============================
    # Preprocess
    # ===============================
    x_train = x_train.astype(np.float32) / 255.
    x_train = x_train.reshape(-1, 28**2)
    x_test = x_test.astype(np.float32) / 255.
    x_test = x_test.reshape(-1, 28**2)
    t_train = to_categorical(t_train, n_classes).astype(np.float32)
    t_test = to_categorical(t_test, n_classes).astype(np.float32)

    # ===============================
    # Instantiate ELM
    # ===============================
    model = ELM(
        n_input_nodes=28**2,
        n_hidden_nodes=args.n_hidden_nodes,
        n_output_nodes=n_classes,
        loss=args.loss,
        activation=args.activation,
        name='elm',
    )

    # ===============================
    # Training
    # ===============================
    model.fit(x_train, t_train)
    train_loss, train_acc, train_uar = model.evaluate(x_train, t_train, metrics=['loss', 'accuracy', 'uar'])
    print('train_loss: %f' % train_loss) # loss value
    print('train_acc: %f' % train_acc) # accuracy
    print('train_uar: %f' % train_uar) # uar (unweighted average recall)

    # ===============================
    # Validation
    # ===============================
    val_loss, val_acc, val_uar = model.evaluate(x_test, t_test, metrics=['loss', 'accuracy', 'uar'])
    print('val_loss: %f' % val_loss)
    print('val_acc: %f' % val_acc)
    print('val_uar: %f' % val_uar)

    # ===============================
    # Prediction
    # ===============================
    x = x_test[:10]
    t = t_test[:10]
    y = softmax(model.predict(x))

    for i in range(len(y)):
        print('---------- prediction %d ----------' % (i+1))
        class_pred = np.argmax(y[i])
        prob_pred = y[i][class_pred]
        class_true = np.argmax(t[i])
        print('prediction:')
        print('\tclass: %d, probability: %f' % (class_pred, prob_pred))
        print('\tclass (true): %d' % class_true)

    # ===============================
    # Save model
    # ===============================
    print('saving model...')
    model.save('model.h5')
    del model

    # ===============================
    # Load model
    # ===============================
    print('loading model...')
    model = load_model('model.h5')


if __name__ == '__main__':
    args = parser.parse_args()
    main(args)


In [0]:
!python train.py --n_hidden_nodes 2048 --activation sigmoid --loss mean_absolute_error

Using TensorFlow backend.
tcmalloc: large alloc 1999716352 bytes == 0x8ae6e000 @  0x7f711bda61e7 0x7f711436c106 0x7f7119b6df4e 0x7f7119b6e37e 0x5aa6ec 0x50abb3 0x50d390 0x508245 0x58958c 0x5a067e 0x7f711995506d 0x50a8af 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x589471 0x5a067e 0x7f711995506d 0x50a8af 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x509d48 0x50aa7d 0x50c5b9 0x509d48
train_loss: 0.088128
train_acc: 0.960783
train_uar: 0.960421
val_loss: 0.090484
val_acc: 0.953200
val_uar: 0.952836
---------- prediction 1 ----------
prediction:
	class: 7, probability: 0.289745
	class (true): 7
---------- prediction 2 ----------
prediction:
	class: 2, probability: 0.195573
	class (true): 2
---------- prediction 3 ----------
prediction:
	class: 1, probability: 0.198742
	class (true): 1
---------- prediction 4 ----------
prediction:
	class: 8, probability: 0.146815
	class (true): 0
---------- prediction 5 ----------
prediction:
	class: 4, probability: 0.194335
	class (true): 4
--

In [0]:
from keras.models import load_model
model = load_model('model.h5')

Using TensorFlow backend.


ValueError: ignored